In [1]:
import os
import requests
import base64
# os.environ['OPENAI_API_KEY'] = 

In [2]:
os.environ['OPENAI_API_KEY'] = 
os.environ['OPENAI_API_TYPE'] = 'azure'
os.environ['OPENAI_API_VERSION'] = '2023-03-15-preview'
os.environ['OPENAI_API_BASE'] = 'https://francecentral-openai.openai.azure.com/'

In [12]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import AzureChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
import re



In [53]:
model_name = 'gpt-35-turbo' #gpt-4

In [54]:
# initialize tokenizer
import tiktoken
enc = tiktoken.encoding_for_model(model_name)

In [55]:
llm = AzureChatOpenAI(
    deployment_name=model_name,
    model_name=model_name,
    temperature=0,
)

/home/issa/miniconda3/envs/LangChain/lib/python3.10/site-packages/langchain_community/chat_models/azure_openai.py:167: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://francecentral-openai.openai.azure.com/ to https://francecentral-openai.openai.azure.com/openai.
  warnings.warn(
/home/issa/miniconda3/envs/LangChain/lib/python3.10/site-packages/langchain_community/chat_models/azure_openai.py:174: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
/home/issa/miniconda3/envs/LangChain/lib/python3.10/site-packages/langchain_community/chat_models/azure_openai.py:182: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) is specifi

In [57]:
# loader = PyPDFLoader("data/rapport-solvabilite-2021-maif-vie.pdf")
# pages = loader.load()#load_and_split()

In [58]:
# # Combine the pages, and replace the tabs with spaces
# text = ""

# for page in pages:
#     text += page.page_content

In [59]:
def transform_func(inputs: dict) -> dict:
    text = inputs["text"]
    
    # replace multiple new lines and multiple spaces with a single one
    text = re.sub(r'(\r\n|\r|\n){2,}', r'\n', text)
    text = re.sub(r'[ \t]+', ' ', text)

    return {"output_text": text}

In [60]:
text_refined = transform_func({'text': text})

## Map Reduce

In [10]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=500)

# docs = text_splitter.create_documents([text_refined['output_text']])

In [61]:
map_prompt = """
Tu es un assistant expert dans le domaine financier et en assurance.

#Règles: 
- Tu ne dois pas modifier les valeurs des données financières (EXP: Chiffres d'affaires, pourcentages, taux, montants, etc)

Ecris moi un résumé détaillé pour le texte suivant:
"{text}"
Résumé:
"""
map_prompt_template = PromptTemplate(template=map_prompt, input_variables=["text"])

In [62]:
combine_prompt = """
Tu es un assistant expert dans le domaine financier et en assurance.

#Règles: 
- Tu ne dois pas modifier les valeurs des données financières (EXP: Chiffres d'affaires, pourcentages, taux, etc)

Ecris moi un résumé du texte suivant avec un titre pour chaque partie du texte:
```{text}```
Résumé:
"""
combine_prompt_template = PromptTemplate(template=combine_prompt, input_variables=["text"])

In [63]:
summary_chain_map_reduce = load_summarize_chain(llm=llm,
                                     chain_type='map_reduce',
                                     map_prompt=map_prompt_template,
                                     combine_prompt=combine_prompt_template,
#                                      verbose=True
                                    )

In [64]:
# text = text_refined['output_text']

In [66]:
def load_and_preprocess(path: str) -> str:
    loader = PyPDFLoader(path)
    pages = loader.load()
    # Combine the pages, and replace the tabs with spaces
    text = ""

    for page in pages:
        text += page.page_content
    
    text_refined = transform_func({'text': text})
    return text_refined['output_text']

In [76]:
def summarize_map_reduce(summary_chain, text:str) -> str:
    #split text into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=500)
    docs = text_splitter.create_documents([text])

    #compute number of tokens before the summarisation
    length_before = len(enc.encode(text=text))
    print (f"length of text before summarization: {length_before} tokens")

    #Generate the summary
    output = summary_chain.run(docs)

    #compute number of tokens after the summarisation
    length_after = len(enc.encode(text=output))
    print (f"length of text after summarization: {length_after} tokens")

    return output


In [77]:
text = load_and_preprocess("data/rapport-solvabilite-2021-maif-vie.pdf")
rapport_n_1 = summarize_map_reduce(summary_chain_map_reduce, text)

length of text before summarization: 76613 tokens
length of text after summarization: 1742 tokens


In [79]:
# print(rapport_n_1)

In [17]:
text = load_and_preprocess("data/rapport-solvabilite-2022-maif-vie.pdf")
rapport_n = summarize_map_reduce(summary_chain_map_reduce, text)

## Résumé direct

In [66]:
# len(enc.encode(prompt_template.format(text=text_refined['output_text'])))

76998

In [80]:
# Define prompt
prompt_template = """Tu es un assistant expert dans le domaine financier et en assurance.

#Règles: 
- Tu ne dois pas modifier les valeurs des données financières (EXP: Chiffres d'affaires, pourcentages, taux, montants, etc)

Ecris moi un résumé détaillé du texte suivant avec un titre pour chaque partie:
{text}
Résumé:
"""
# we set the prompt to only have the question we ask
prompt = PromptTemplate(input_variables=['text'], template=prompt_template)

# Define LLM chain
llm_chain = LLMChain(llm=llm, prompt=prompt)

res = llm_chain.run(text=text_refined['output_text'])

## Comparaison

In [39]:
# Define prompt
prompt_template = """Tu es un assistant expert dans le domaine financier et en assurance. Tu sais identifier des potentiels
risques dans l'activité d'un organisme. Tu ne peux pas faire des calculs mathématiques complexes sans te servir d'un outil adapté.
Tu as à disposition deux rapport SFCR (solvabilité et sécurité financière) sur l'activité d'une entreprise de deux années
consécutives 'N' et 'N-1'. Ta tâche consiste à générer une fiche de synthèse comparative entre les deux rapports fournis sous 
le format suivant:
'''
1.Présentation []
2.Activité
3.Placements
4.Gouvernance
5.Perspectives
'''

#Règles: 
- Si tu ne trouves pas l'information sur une partie donnée tu peux l'ignorer.
- Si dans le rapport de l'année N il y déjà une comparaison avec l'année N-1 tu dois la garder.

#Rapport de l'année 'N-1'
{rapport_n_1}

#Rapport de l'année 'N'
{rapport_n}

Génération de la fiche synthèse:
1.Présentation
"""
# we set the prompt to only have the question we ask
prompt = PromptTemplate(input_variables=['rapport_n_1','rapport_n'], template=prompt_template)

# Define LLM chain
llm_chain = LLMChain(llm=llm, prompt=prompt)

# res = llm_chain.run(
#     rapport_n_1=rapport_n_1,
#     rapport_n=rapport_n
#     )

In [23]:
from langchain.chains import LLMMathChain
llm_math = LLMMathChain(llm=llm, verbose=True)

# llm_math.run("What is 13 raised to the .3432 power?")

/home/issa/miniconda3/envs/LangChain/lib/python3.10/site-packages/langchain/chains/llm_math/base.py:57: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [24]:
from langchain.agents import Tool

tools = [
    Tool(
        name='outil de Mathématiques',
        func=llm_math.run,
        description=(
            'utilise cet outil lorsque tu veux faire un calcul mathématique'
        )
    )
]

In [50]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
# conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)

In [51]:
from langchain.agents import initialize_agent

agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory
)

In [48]:
prompt.format_prompt(rapport_n= output, rapport_n_1= output)

StringPromptValue(text="Tu es un assistant expert dans le domaine financier et en assurance. Tu sais identifier des potentiels\nrisques dans l'activité d'un organisme. Tu ne peux pas faire des calculs mathématiques complexes sans te servir d'un outil adapté.\nTu as à disposition deux rapport SFCR (solvabilité et sécurité financière) sur l'activité d'une entreprise de deux années\nconsécutives 'N' et 'N-1'. Ta tâche consiste à générer une fiche de synthèse comparative entre les deux rapports fournis sous \nle format suivant:\n'''\n1.Présentation []\n2.Activité\n3.Placements\n4.Gouvernance\n5.Perspectives\n'''\n\n#Règles: \n- Si tu ne trouves pas l'information sur une partie donnée tu peux l'ignorer.\n- Si dans le rapport de l'année N il y déjà une comparaison avec l'année N-1 tu dois la garder.\n\n#Rapport de l'année 'N-1'\nGestion des risques financiers et opérationnels\nLe rapport met en lumière la gestion des risques financiers et opérationnels de MAIF VIE pour l'année 2021, abordant

In [52]:
agent(prompt.format_prompt(rapport_n= output, rapport_n_1= output).text)



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "La génération de la fiche synthèse comparative entre les deux rapports SFCR nécessite une analyse approfondie des informations fournies dans les rapports de l'année 'N' et 'N-1'. Il est important de comparer les éléments clés tels que la gestion des risques, les mesures d'atténuation, l'engagement en faveur de la transition énergétique et écologique, les provisions techniques, les outils de modélisation, la valorisation et comptabilisation des impôts différés, la gestion des fonds propres, la politique de dividende, les provisions techniques et engagements d'assurance, les exigences de capital de solvabilité, ainsi que les fonds propres éligibles et disponibles. Une synthèse comparative peut ensuite être établie en mettant en évidence les différences et similitudes entre les deux années, en accordant une attention particulière aux évolutions significatives et aux tendances émergentes."


{'input': "Tu es un assistant expert dans le domaine financier et en assurance. Tu sais identifier des potentiels\nrisques dans l'activité d'un organisme. Tu ne peux pas faire des calculs mathématiques complexes sans te servir d'un outil adapté.\nTu as à disposition deux rapport SFCR (solvabilité et sécurité financière) sur l'activité d'une entreprise de deux années\nconsécutives 'N' et 'N-1'. Ta tâche consiste à générer une fiche de synthèse comparative entre les deux rapports fournis sous \nle format suivant:\n'''\n1.Présentation []\n2.Activité\n3.Placements\n4.Gouvernance\n5.Perspectives\n'''\n\n#Règles: \n- Si tu ne trouves pas l'information sur une partie donnée tu peux l'ignorer.\n- Si dans le rapport de l'année N il y déjà une comparaison avec l'année N-1 tu dois la garder.\n\n#Rapport de l'année 'N-1'\nGestion des risques financiers et opérationnels\nLe rapport met en lumière la gestion des risques financiers et opérationnels de MAIF VIE pour l'année 2021, abordant la constitut